In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

  # Install the latest version of torchtext library quietly without showing output


!pip install transformers evaluate wandb datasets accelerate peft bitsandbytes -U -qq ## NEW LINES ##
basepath = '/content/drive/MyDrive/NLP/'

Mounted at /content/drive


In [2]:
from pathlib import Path
# wandb library
import wandb
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
from sklearn import metrics
import torch
torch.cuda.empty_cache()
#from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
import os, sys
os.environ['TOKENIZERS_PARALLELISM']='false'

from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import transformers
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, RobertaForSequenceClassification,AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, set_seed
from transformers import T5ForSequenceClassification, AutoTokenizer
from transformers import AutoConfig


from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets
from datasets import Dataset, DatasetDict

import evaluate



!git lfs install

Git LFS initialized.


In [3]:
from peft import (
    TaskType,
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
)

In [4]:
base_folder = Path(basepath)
data_folder = base_folder/'Homework/Homework7'
model_folder = base_folder/'models/nlp_spring_2024/ed/nn'
custom_functions = base_folder/'custom-functions'

In [5]:
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')
import CustomPreprocessorSpacy as cp

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [8]:
train_data = pd.read_csv(data_folder/ 'train.csv')
test= pd.read_csv(data_folder/ 'test.csv')

In [9]:
train_data.drop(columns=["ID"],inplace=True)
y_test = test.drop(['ID','Tweet'],axis=1)
for column in y_test:
    y_test[column] = y_test[column].apply(lambda x: 0 if x == "NONE" else 1)
y_test = y_test.apply(lambda x: list(map(int, x)))
test_data = pd.concat([test['Tweet'],y_test],axis=1)

train_data.head()


,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


In [10]:
test_data.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,0,0,0,0,0,0,0,0,0,0,0
1,"Academy of Sciences, eschews the normally sobe...",0,0,0,0,0,0,0,0,0,0,0
2,I blew that opportunity -__- #mad,0,0,0,0,0,0,0,0,0,0,0
3,This time in 2 weeks I will be 30... 😥,0,0,0,0,0,0,0,0,0,0,0
4,#Deppression is real. Partners w/ #depressed p...,0,0,0,0,0,0,0,0,0,0,0


In [11]:
labels = [x for x in train_data.columns if x not in ['Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}

label2id = {label:idx for idx, label in enumerate(labels)}
NUM_OF_LABELS = len(labels)

In [12]:
labels = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# Creating a new DataFrame with the 'tweet' column followed by the emotions columns as a list
train_df = pd.DataFrame({'text': train_data['Tweet'], 'label': train_data[labels].values.tolist()})
test_df = pd.DataFrame({'text': test_data['Tweet'], 'label': test_data[labels].values.tolist()})
train_df.head()

,text,label
0,“Worry is a down payment on a problem you may ...,"[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]"
1,Whatever you decide to do make sure it makes y...,"[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]"
2,@Max_Kellerman it also helps that the majorit...,"[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]"
3,Accept the challenges so that you can literall...,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]"
4,My roommate: it's okay that we can't spell bec...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"


In [13]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_testEval = train_dataset.train_test_split(train_size=0.80)
val_eval = train_testEval['test']

ds = DatasetDict({
    'train' : train_testEval['train'],
    'val' : val_eval,
    'test':test_dataset

})

print("Training Dataset Shape:", ds['train'].shape)
print("Evaluation Dataset Shape:", ds['val'].shape)
print("Testing Dataset Shape",ds['test'].shape)
ds['train'][25]

Training Dataset Shape: (6179, 2)
Evaluation Dataset Shape: (1545, 2)
Testing Dataset Shape (3259, 2)


{'text': 'If you be #angry, which may be the case, &amp; which may be unavoidable, see that the sudden excitement does not become sin. #AlbertBarnes',
 'label': [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]}

In [14]:
checkpoint = "Alibaba-NLP/gte-Qwen1.5-7B-instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
def tokenize_fn(batch):
    return tokenizer(text = batch["text"], truncation=True)

In [16]:
tokenized_dataset= ds.map(tokenize_fn, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['text']
)
# tokenized_dataset.set_format(type='torch')


Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 6179
    })
    val: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1545
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3259
    })
})

In [18]:
accuracy_metric = evaluate.load('accuracy', 'multilabel')
f1 = evaluate.load('f1','multilabel')

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    #logits= logits[0]

    # print(logits.shape)
    preds = (logits > 0).astype(int)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1_micro = f1.compute(predictions=preds, references=labels, average='micro')
    f1_macro = f1.compute(predictions=preds, references=labels, average='macro')
    return {'f1_micro':f1_micro['f1'],
            'f1_macro':f1_macro['f1'],
            'accuracy':accuracy['accuracy'],
            }

In [19]:
# Define the directory where model checkpoints will be saved
run_name = "emotions_Qwen1.5_qlora_im"
base_folder = Path(basepath)
model_folder = base_folder / "models"/run_name
# Create the directory if it doesn't exist
model_folder.mkdir(exist_ok=True, parents=True)

# Configure training parameters
training_args = TrainingArguments(
    # Training-specific configurations
    num_train_epochs=10,  # Total number of training epochs
    # Number of samples per training batch for each device
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    # gradient_accumulation_steps=8,

    weight_decay=0.001,  # Apply L2 regularization to prevent overfitting
    learning_rate=5e-6,  # Step size for the optimizer during training
    lr_scheduler_type='linear',
    warmup_steps=0,  # Number of warmup steps for the learning rate scheduler
    optim='adamw_torch',  # Optimizer,
    max_grad_norm = 1.0,

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=20,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=20,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=1,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations (commented out in this example)
    logging_strategy='steps',
    logging_steps=200,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name=run_name,  # Experiment name for Weights & Biases

    fp16=True,
    #bf16=True
    # tf32= False
)


In [20]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  llm_int8_skip_modules = ['score'],
  bnb_4bit_quant_type="nf4",
  bnb_4bit_use_double_quant=True,
  bnb_4bit_compute_dtype=torch.bfloat16
)

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=11,
                                                           problem_type="multi_label_classification",
                                                           quantization_config=bnb_config)

model = prepare_model_for_kbit_training(model)

config = AutoConfig.from_pretrained(checkpoint)
id2label= {id_: label_ for id_, label_ in enumerate(labels)}
label2id = {label_: id_ for id_, label_ in enumerate(labels)}
config.id2label = id2label
config.label2id = label2id
model.config = config
model.config.pad_token_id = tokenizer.pad_token_id
model


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Alibaba-NLP/gte-Qwen1.5-7B-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Qwen2ForSequenceClassification(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-31): 32 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=True)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=True)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=True)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
   

In [22]:
model.config

Qwen2Config {
  "_name_or_path": "Alibaba-NLP/gte-Qwen1.5-7B-instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoModel": "Alibaba-NLP/gte-Qwen1.5-7B-instruct--modeling_qwen.Qwen2Model",
    "AutoModelForCausalLM": "Alibaba-NLP/gte-Qwen1.5-7B-instruct--modeling_qwen.Qwen2ForCausalLM",
    "AutoModelForSequenceClassification": "Alibaba-NLP/gte-Qwen1.5-7B-instruct--modeling_qwen.Qwen2ForSequenceClassification"
  },
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimis

In [23]:
import re
model_modules = str(model.modules)
pattern = r'\((\w+)\): Linear'
linear_layer_names = re.findall(pattern, model_modules)

names = []
# Print the names of the Linear layers
for name in linear_layer_names:
    names.append(name)
target_modules = list(set(names))
target_modules

['up_proj',
 'o_proj',
 'k_proj',
 'gate_proj',
 'down_proj',
 'v_proj',
 'q_proj',
 'score']

In [24]:
gemma_peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=100,
    lora_alpha=200,
    lora_dropout=0.01,
    bias="lora_only",
    modules_to_save = ['score'],
    target_modules = ['v_proj','up_proj','k_proj','o_proj','q_proj','score','down_proj','gate_proj']
)
gemma_peft_model = get_peft_model(model, gemma_peft_config )
gemma_peft_model.print_trainable_parameters()

trainable params: 250,294,272 || all params: 7,348,940,800 || trainable%: 3.405855058731729


In [25]:
gemma_peft_config.target_modules

{'down_proj',
 'gate_proj',
 'k_proj',
 'o_proj',
 'q_proj',
 'score',
 'up_proj',
 'v_proj'}

In [26]:
gemma_peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Qwen2ForSequenceClassification(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 4096)
        (layers): ModuleList(
          (0-31): 32 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.01, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=100, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=100, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
       

In [27]:
def calculate_pos_weights(dataset):
    # Initialize counters for all labels
    num_labels = len(dataset['train']['label'][0])
    total_positives = [0] * num_labels
    total_negatives = [0] * num_labels

    # Count positives and negatives for each label
    for label_array in dataset['train']['label']:
        for i, label in enumerate(label_array):
            if label == 1:
                total_positives[i] += 1
            else:
                total_negatives[i] += 1

    # Calculate pos_weight for each label
    pos_weight = [total_negatives[i] / max(total_positives[i], 1) for i in range(num_labels)]
    return torch.tensor(pos_weight)

# Calculate the pos_weight using the training set
pos_weights = calculate_pos_weights(ds)


In [28]:
pos_weights

tensor([ 1.7077,  6.0216,  1.6338,  4.7372,  1.6588,  8.3621,  2.3636,  7.5228,
         2.3876, 18.4921, 18.4308])

In [29]:
pos_weights= torch.tensor([2., 3., 2., 2., 2., 3., 2., 3., 2., 4., 4.])

In [30]:
import torch.nn as nn
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").float()  # Ensure labels are float for BCE loss
        outputs = model(**inputs)
        logits = outputs.get("logits")

        device = next(model.parameters()).device

        loss_fct = nn.BCEWithLogitsLoss(pos_weight=pos_weights.to(device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [31]:
trainer = CustomTrainer(
    model=gemma_peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [32]:
wandb.login()

wandb: Currently logged in as: rangareddy-bhargav. Use `wandb login --relogin` to force relogin


True

In [33]:
trainer.train()  # start training

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
20,No log,1.773256,0.341897,0.270600,0.005178
40,No log,1.186590,0.380881,0.293129,0.009709
60,No log,0.860986,0.430044,0.317386,0.029773
80,No log,0.722414,0.515557,0.358865,0.067314
100,No log,0.639634,0.578762,0.442441,0.099676
120,No log,0.602684,0.611167,0.487430,0.109385
140,No log,0.580673,0.626300,0.496714,0.115858
160,No log,0.556731,0.650192,0.518934,0.177994
180,No log,0.547822,0.651991,0.532823,0.175405
200,0.962600,0.531694,0.665898,0.535702,0.182524


TrainOutput(global_step=970, training_loss=0.44429533457018666, metrics={'train_runtime': 9843.922, 'train_samples_per_second': 6.277, 'train_steps_per_second': 0.099, 'total_flos': 1.2104404552153498e+17, 'train_loss': 0.44429533457018666, 'epoch': 10.0})

In [34]:
trainer.evaluate()

{'eval_loss': 0.5000859498977661,
 'eval_f1_micro': 0.6933333333333334,
 'eval_f1_macro': 0.6042017181890457,
 'eval_accuracy': 0.2297734627831715,
 'eval_runtime': 53.0259,
 'eval_samples_per_second': 29.137,
 'eval_steps_per_second': 0.471,
 'epoch': 10.0}

In [35]:
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 460.


In [36]:
model_folder

PosixPath('/content/drive/MyDrive/NLP/models/emotions_Qwen1.5_qlora_im')

In [37]:
model_checkpoint = str(model_folder) + '/checkpoint-' + best_model_checkpoint_step

In [38]:
valid_results = trainer.evaluate(tokenized_dataset["val"])

In [39]:
valid_results

{'eval_loss': 0.5000859498977661,
 'eval_f1_micro': 0.6933333333333334,
 'eval_f1_macro': 0.6042017181890457,
 'eval_accuracy': 0.2297734627831715,
 'eval_runtime': 53.2391,
 'eval_samples_per_second': 29.02,
 'eval_steps_per_second': 0.47,
 'epoch': 10.0}

In [40]:
gemma_peft_model.save_pretrained('/content/drive/MyDrive/NLP/testing')

In [41]:
from huggingface_hub import notebook_login
notebook_login()

In [42]:
gemma_peft_model.push_to_hub("zura1101/Qwen_peft_model_emotion_detection_qlora")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zura1101/Qwen_peft_model_emotion_detection_qlora/commit/e8c51b5603573c1e418ab770a6363ba49fa44ae7', commit_message='Upload model', commit_description='', oid='e8c51b5603573c1e418ab770a6363ba49fa44ae7', pr_url=None, pr_revision=None, pr_num=None)

In [43]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer

peft_model_id = "zura1101/Qwen_peft_model_emotion_detection_qlora"
config = PeftConfig.from_pretrained(peft_model_id)
config.base_model_name_or_path
base_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path,
                                                                num_labels=11,
                                                                problem_type="multi_label_classification",
                                                                quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-1.1-2b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
# Load the Lora model
inference_model = PeftModel.from_pretrained(base_model, peft_model_id)

In [45]:
inference_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GemmaForSequenceClassification(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.01, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=100, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=100, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
   

In [46]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to=[],
    fp16 = True
)

In [47]:
trainer = CustomTrainer(
    model=inference_model,
    args=training_args,
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [48]:
test_results = trainer.predict(tokenized_dataset["test"])

# The predictions object contains various fields including predictions
logits = test_results.predictions

# Apply sigmoid to get the probabilities
probs = torch.sigmoid(torch.Tensor(logits))

# Apply a threshold to get the binary predictions
threshold = 0.5
y_pred = (probs >= threshold).int().numpy()

In [49]:
y_pred

array([[0, 1, 0, ..., 0, 1, 1],
       [0, 1, 1, ..., 1, 1, 0],
       [0, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int32)

In [50]:
y_pred_df = pd.DataFrame(y_pred,columns=labels)
output_submission = pd.concat([test['ID'],y_pred_df],axis=  1)
output_submission

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,0,1,0,0,0,0,1,0,0,1,1
1,2018-03739,0,1,1,0,1,0,1,0,1,1,0
2,2018-00385,0,1,0,1,0,0,1,0,0,0,1
3,2018-03001,0,1,0,0,1,0,1,0,0,0,0
4,2018-01988,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,1,1,0,0,0,0,0,0,0,0,0
3255,2018-00416,0,1,0,0,1,0,0,0,0,0,0
3256,2018-03717,0,1,0,0,0,0,1,0,0,0,0
3257,2018-03504,0,1,0,0,0,0,1,0,0,0,1


In [51]:
output_submission.to_csv('/content/drive/MyDrive/NLP/output_submission.csv', index=False)

In [52]:
wandb.finish()

eval/accuracy,▁▁▂▃▄▄▆▆▇▇▆▆█▆▇▇▆████▇█▇███▇█▇██▇███████
eval/f1_macro,▁▁▂▃▆▆▆▆▇▇▇██████████████████▇▇▇▇▇▇▇▇▇▇█
eval/f1_micro,▁▂▃▄▆▇▇▇██████████████████████████▇█▇▇██
eval/loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▁
eval/runtime,▅▅▅▄▄▁▃▅▄▂▃▂▅▃▄▅▃▆▆▅▄▆▄▅▄▄▄▄▅▅▅█▇▄▇▄██▅█
eval/samples_per_second,▄▄▄▅▅█▇▄▅▇▇▇▄▆▅▄▆▃▃▄▅▃▅▄▅▅▅▅▄▄▄▁▂▅▂▅▁▁▄▁
eval/steps_per_second,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,▁█▅█
train/learning_rate,█▆▃▁
